In [1]:
import requests
import pandas as pd
from matplotlib import pyplot as plt
import seaborn as sns

In [2]:
query="2022-04-26T20:00:00"

## Define functions

Write the functions to pull data from the APIs

In [3]:
def get_weather_data(query:str):
    data_sets = ["air-temperature","rainfall","relative-humidity"]
    results=[]
    for measure in data_sets:
        URL = "https://api.data.gov.sg/v1/environment/"+measure
        params={'date_time':query}
        r=requests.get(URL,params=params)
        results.append(r.json())
    return results

def get_taxi_data(query:str):
    URL = "https://api.data.gov.sg/v1/transport/taxi-availability"
    params={'date_time':query}
    r=requests.get(URL,params=params)
    return r.json()

### Pull data

TODO: 

Assign each coordinate to nearest Euclidean distance weather station

Count availability for each weather station

In [4]:
weather_data=get_weather_data(query=query)
taxi_data=get_taxi_data(query=query)

In [7]:
taxi_data['type']

'FeatureCollection'

In [8]:
taxi_data['crs']

{'type': 'link',
 'properties': {'href': 'http://spatialreference.org/ref/epsg/4326/ogcwkt/',
  'type': 'ogcwkt'}}

In [9]:
taxi_data['features']

[{'type': 'Feature',
  'geometry': {'type': 'MultiPoint',
   'coordinates': [[103.62223, 1.29],
    [103.62651, 1.29535],
    [103.66389, 1.32283],
    [103.66588, 1.32207],
    [103.68659, 1.34295],
    [103.68749, 1.34],
    [103.688634, 1.34119983333333],
    [103.692578383333, 1.34290998333333],
    [103.69418, 1.34],
    [103.69426, 1.35],
    [103.694441416667, 1.34384483333333],
    [103.696176666667, 1.34535683333333],
    [103.696731916667, 1.34826105],
    [103.696793966667, 1.34003755],
    [103.697088, 1.34880583333333],
    [103.69725, 1.34063],
    [103.698209533333, 1.33501028333333],
    [103.69864685, 1.35583136666667],
    [103.698813433333, 1.31161401666667],
    [103.699957333333, 1.34744266666667],
    [103.70069, 1.35],
    [103.7012, 1.33882],
    [103.7014085, 1.337115],
    [103.701417066667, 1.34724893333333],
    [103.70187, 1.35982],
    [103.703418666667, 1.35225083333333],
    [103.70389, 1.35],
    [103.70391, 1.36],
    [103.70407, 1.33],
    [103.70497,

In [10]:
weather_data[0]

{'metadata': {'stations': [{'id': 'S109',
    'device_id': 'S109',
    'name': 'Ang Mo Kio Avenue 5',
    'location': {'latitude': 1.3764, 'longitude': 103.8492}},
   {'id': 'S50',
    'device_id': 'S50',
    'name': 'Clementi Road',
    'location': {'latitude': 1.3337, 'longitude': 103.7768}},
   {'id': 'S107',
    'device_id': 'S107',
    'name': 'East Coast Parkway',
    'location': {'latitude': 1.3135, 'longitude': 103.9625}},
   {'id': 'S43',
    'device_id': 'S43',
    'name': 'Kim Chuan Road',
    'location': {'latitude': 1.3399, 'longitude': 103.8878}},
   {'id': 'S108',
    'device_id': 'S108',
    'name': 'Marina Gardens Drive',
    'location': {'latitude': 1.2799, 'longitude': 103.8703}},
   {'id': 'S44',
    'device_id': 'S44',
    'name': 'Nanyang Avenue',
    'location': {'latitude': 1.34583, 'longitude': 103.68166}},
   {'id': 'S121',
    'device_id': 'S121',
    'name': 'Old Choa Chu Kang Road',
    'location': {'latitude': 1.37288, 'longitude': 103.72244}},
   {'id': '

In [11]:
weather_data[1]

{'metadata': {'stations': [{'id': 'S77',
    'device_id': 'S77',
    'name': 'Alexandra Road',
    'location': {'latitude': 1.2937, 'longitude': 103.8125}},
   {'id': 'S109',
    'device_id': 'S109',
    'name': 'Ang Mo Kio Avenue 5',
    'location': {'latitude': 1.3764, 'longitude': 103.8492}},
   {'id': 'S90',
    'device_id': 'S90',
    'name': 'Bukit Timah Road',
    'location': {'latitude': 1.3191, 'longitude': 103.8191}},
   {'id': 'S114',
    'device_id': 'S114',
    'name': 'Choa Chu Kang Avenue 4',
    'location': {'latitude': 1.38, 'longitude': 103.73}},
   {'id': 'S50',
    'device_id': 'S50',
    'name': 'Clementi Road',
    'location': {'latitude': 1.3337, 'longitude': 103.7768}},
   {'id': 'S107',
    'device_id': 'S107',
    'name': 'East Coast Parkway',
    'location': {'latitude': 1.3135, 'longitude': 103.9625}},
   {'id': 'S215',
    'device_id': 'S215',
    'name': 'GEYLANG EAST CENTRAL',
    'location': {'latitude': 1.32785, 'longitude': 103.88899}},
   {'id': 'S118

In [12]:
weather_data[2]

{'metadata': {'stations': [{'id': 'S109',
    'device_id': 'S109',
    'name': 'Ang Mo Kio Avenue 5',
    'location': {'latitude': 1.3764, 'longitude': 103.8492}},
   {'id': 'S50',
    'device_id': 'S50',
    'name': 'Clementi Road',
    'location': {'latitude': 1.3337, 'longitude': 103.7768}},
   {'id': 'S107',
    'device_id': 'S107',
    'name': 'East Coast Parkway',
    'location': {'latitude': 1.3135, 'longitude': 103.9625}},
   {'id': 'S43',
    'device_id': 'S43',
    'name': 'Kim Chuan Road',
    'location': {'latitude': 1.3399, 'longitude': 103.8878}},
   {'id': 'S108',
    'device_id': 'S108',
    'name': 'Marina Gardens Drive',
    'location': {'latitude': 1.2799, 'longitude': 103.8703}},
   {'id': 'S44',
    'device_id': 'S44',
    'name': 'Nanyang Avenue',
    'location': {'latitude': 1.34583, 'longitude': 103.68166}},
   {'id': 'S121',
    'device_id': 'S121',
    'name': 'Old Choa Chu Kang Road',
    'location': {'latitude': 1.37288, 'longitude': 103.72244}},
   {'id': '

# Modeling

Try different time lag for forecasting availability. 1 min, 15 min, 30 min etc.

# Serving Prediction

TODO: Decide on what we are trying to predict